In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/CVAssignment2/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import cv2

train_x = np.load(root_path+"Pawan/Split/train_x.npy")
train_y = np.load(root_path+"Pawan/Split/train_y.npy")
test_x = np.load(root_path+"Pawan/Split/test_x.npy")
test_y = np.load(root_path+"Pawan/Split/test_y.npy")



print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1854, 512, 512, 3)
(1854, 3)
(464, 512, 512, 3)
(464, 3)


In [0]:
## Model Architecture
import tensorflow as tf
from keras import backend as K
from keras import backend
from keras import models

from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Conv2DTranspose, Activation, add, multiply,Lambda, UpSampling2D, concatenate
import os
from keras import Model
from keras.layers.normalization import BatchNormalization
from keras.applications import imagenet_utils
from keras import layers
from keras.applications.imagenet_utils import decode_predictions

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same')(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer='glorot_uniform', name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x

def expend_as(tensor, rep,name):
    my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
    return my_repeat


def dense_layer(y, blocks, name):
    for i in range(blocks):
        y = conv_layer(y, 32, name=name + '_block' + str(i + 1))
    return y


def transition_layer(y, reduction, name):
    bn_axis = 3 
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(y)
    y = layers.Activation('relu', name=name + '_relu')(y)
    y = layers.Conv2D(int(backend.int_shape(y)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(y)
    y = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(y)
    return y


def conv_layer(y, growth_rate, name):
    bn_axis = 3 
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(y)
    y1 = layers.Activation('relu', name=name + '_0_relu')(y1)
    y1 = layers.SeparableConv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(y1)
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(y1)
    y1 = layers.Activation('relu', name=name + '_1_relu')(y1)
    y1 = layers.SeparableConv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(y1)
    y = layers.Concatenate(axis=bn_axis, name=name + '_concatenate')([y, y1])
    return y

def Model1():
    bn_axis = 3

    image=layers.Input((512,512,3))
    y = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(image)
    y = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(y)
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(y)
    y = layers.Activation('relu', name='conv1/relu')(y)
    y = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(y)
    y = layers.MaxPooling2D(3, strides=2, name='pool1')(y)

    fire2_squeeze = Conv2D(16, 1, 1, activation='relu', init='glorot_uniform',border_mode='same', name='fire2_squeeze')(y)
    fire2_expand1 = Conv2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same', name='fire2_expand1')(fire2_squeeze)
    fire2_expand2 = Conv2D(64, 3, 3, activation='relu', init='glorot_uniform',border_mode='same', name='fire2_expand2')(fire2_squeeze)
    merge = layers.Concatenate(axis=bn_axis)([fire2_expand1, fire2_expand2])
    merge = layers.Concatenate(axis=bn_axis)([merge, y])

    y = dense_layer(merge, 6, name='conv2')
    y = transition_layer(y, 0.5, name='pool2')
    y = dense_layer(y, 12, name='conv3')
    y = transition_layer(y, 0.5, name='pool3')
    y = dense_layer(y, 24, name='conv4')
    y = transition_layer(y, 0.5, name='pool4')
    y = dense_layer(y, 16, name='conv5')

    fire2_squeeze = Conv2D(64, 1, 1, activation='relu', init='glorot_uniform',border_mode='same', name='firen_squeeze')(y)
    fire2_expand1 = Conv2D(256, 1, 1, activation='relu', init='glorot_uniform',border_mode='same', name='firen_expand1')(fire2_squeeze)
    fire2_expand2 = Conv2D(256, 3, 3, activation='relu', init='glorot_uniform',border_mode='same', name='firen_expand2')(fire2_squeeze)
    merge = layers.Concatenate(axis=bn_axis)([fire2_expand1, fire2_expand2])
    merge = layers.Concatenate(axis=bn_axis)([merge, y])

    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(merge)
    y = layers.Activation('relu', name='relu')(y)
    y2 = GlobalAveragePooling2D()(y)
    y3 = Dropout(0.3)(y2)
    output1 = Dense(10,activation='relu')(y3)
    output2 = Dense(3,activation='softmax')(output1)

    model = models.Model( image, output2, name='model_1')
    return model

from keras.optimizers import SGD, Adam, Adadelta
model=Model1()
model.compile(optimizer=Adam(lr=3e-4), loss=weighted_categorical_crossentropy([0.1723,1.0,1.0117]), metrics=['accuracy'])
model.summary()

In [0]:
# Model Architecture
import tensorflow as tf
from keras import backend as K
from keras import backend
from keras import models
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Conv2DTranspose, Activation, add, multiply,Lambda, UpSampling2D, concatenate
import os
from keras import Model
from keras.layers.normalization import BatchNormalization
from keras.applications import imagenet_utils
from keras import layers
from keras.applications.imagenet_utils import decode_predictions

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same')(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer='glorot_uniform', name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x

def expend_as(tensor, rep,name):
    my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
    return my_repeat


def dense_layer(y, blocks, name):
    for i in range(blocks):
        y = conv_layer(y, 32, name=name + '_block' + str(i + 1))
    return y


def transition_layer(y, reduction, name):
    bn_axis = 3 
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(y)
    y = layers.Activation('relu', name=name + '_relu')(y)
    y = layers.Conv2D(int(backend.int_shape(y)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(y)
    y = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(y)
    return y


def conv_layer(y, growth_rate, name):
    bn_axis = 3 
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(y)
    y1 = layers.Activation('relu', name=name + '_0_relu')(y1)
    y1 = layers.SeparableConv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(y1)
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(y1)
    y1 = layers.Activation('relu', name=name + '_1_relu')(y1)
    y1 = layers.SeparableConv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(y1)
    y = layers.Concatenate(axis=bn_axis, name=name + '_concatenate')([y, y1])
    return y

def Model1():
    bn_axis = 3

    image=layers.Input((512,512,3))
    y = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(image)
    y = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(y)
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(y)
    y = layers.Activation('relu', name='conv1/relu')(y)
    y = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(y)
    y = layers.MaxPooling2D(3, strides=2, name='pool1')(y)

    y = dense_layer(y, 4, name='conv2')
    y = transition_layer(y, 0.5, name='pool2')
    y1 = dense_layer(y, 8, name='conv3')
    y = transition_layer(y1, 0.5, name='pool3')
    g2 = UnetGatingSignal(y, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(y1, g2, 128, '_2')
    y = concatenate([Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer='glorot_uniform')(y), attn2], name='up2')
    y = dense_layer(y, 16, name='conv4')
    y = transition_layer(y, 0.5, name='pool4')
    y = dense_layer(y, 12, name='conv5')

    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(y)
    y = layers.Activation('relu', name='relu')(y)
    y2 = GlobalAveragePooling2D()(y)
    y3 = Dropout(0.3)(y2)
    output1 = Dense(10,activation='relu')(y3)
    output2 = Dense(3,activation='softmax')(output1)

    model = models.Model(image , output2, name='model_1')
    return model

from keras.optimizers import SGD, Adam, Adadelta
model=Model1()
model.compile(optimizer=Adam(lr=3e-4), loss=weighted_categorical_crossentropy([0.1723,1.0,1.0117]), metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:

import math
from keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = ModelCheckpoint(root_path+'Logs/rbg_best0007.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
cv = CSVLogger(root_path+'Logs/rbg_best0007.csv', separator=',', append=False)

In [0]:
model.fit(train_x, train_y, batch_size=8, epochs=100, validation_split=(0.2),verbose=1,callbacks=[es,mc,cv])

Train on 1483 samples, validate on 371 samples
Epoch 1/100
1483/1483 [==============================] - 271s 183ms/step - loss: 0.4422 - accuracy: 0.7782 - val_loss: 0.9940 - val_accuracy: 0.4663

Epoch 00001: val_loss improved from inf to 0.99397, saving model to gdrive/My Drive/CVAssignment2/Logs/rbg_best0007.h5
Epoch 2/100
1483/1483 [==============================] - 227s 153ms/step - loss: 0.3065 - accuracy: 0.8348 - val_loss: 1.2396 - val_accuracy: 0.4663

Epoch 00002: val_loss did not improve from 0.99397
Epoch 3/100
1483/1483 [==============================] - 227s 153ms/step - loss: 0.2987 - accuracy: 0.8456 - val_loss: 1.1633 - val_accuracy: 0.4663

Epoch 00003: val_loss did not improve from 0.99397
Epoch 4/100
1483/1483 [==============================] - 227s 153ms/step - loss: 0.2485 - accuracy: 0.8550 - val_loss: 7.6185 - val_accuracy: 0.4663

Epoch 00004: val_loss did not improve from 0.99397
Epoch 5/100
1483/1483 [==============================] - 227s 153ms/step - loss: 

In [0]:
model.save(root_path+'Logs/rbg_last0007.h5')

In [0]:
model.load_weights(root_path+'Logs/rbg_best0007.h5')


In [0]:
test_pred = model.predict(test_x, batch_size=8)

In [0]:
ftest = np.zeros(len(test_pred))
atest = np.zeros(len(test_y))

In [0]:
for i in range(len(test_y)):
  if test_y[i][0] >= test_y[i][1] and test_y[i][0] >= test_y[i][2]:
    atest[i] = 0
  elif test_y[i][1] >= test_y[i][0] and test_y[i][1] >= test_y[i][2]:
    atest[i] = 1
  else:
    atest[i] = 2


In [0]:
for i in range(len(test_pred)):
  if test_pred[i][0] >= test_pred[i][1] and test_pred[i][0] >= test_pred[i][2]:
    ftest[i] = 0
  elif test_pred[i][1] >= test_pred[i][0] and test_pred[i][1] >= test_pred[i][2]:
    ftest[i] = 1
  else:
    ftest[i] = 2

In [0]:
from sklearn import metrics
print(metrics.confusion_matrix(atest, ftest))

In [0]:
print(metrics.classification_report(atest, ftest, digits=3))

In [0]:
# 5 is attention plus squeeze plus densenet

In [0]:
# 6 is squeeze plus densenet

In [0]:
# 7 is attention plus densenet increased values